In [1]:
#initialise dependencies


from matplotlib import pyplot as plt
from numpy.linalg import norm
import numpy as np

from scipy import sparse



from pathlib import Path


from os import listdir
import os
import glob
from PIL import Image
import rawpy
import imageio


from PIL import Image
import PIL

from PIL.ExifTags import TAGS
import imutils

import os
#import hcluster

import re
from datetime import datetime
import pandas as pd
import time

from math import pow
import scipy.signal 
%matplotlib qt
#template matching

import cv2 as cv
import numpy as np

import os
import sys
from tqdm.notebook import tqdm
from time import sleep


In [7]:
#General Functions
def listdirs(folder):
## a function which lists the files in a folder and adds to a list. returns a list of folders, 
##input folder: the file path to folder
    return [
        d for d in (os.path.join(folder, d1) for d1 in os.listdir(folder))
        if os.path.isdir(d)
    ]

def makeclippingmask(image):
    #makes a clipping mask around each bright spot so the analysis isn't thrown off
    grayA = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    blurred = cv.GaussianBlur(grayA, (11, 11), 0)
    thresh = cv.threshold(blurred, 60, 255, cv.THRESH_BINARY)[1]
    #plt.imshow(thresh)
    #ret, thresh = cv.threshold(blurred, 127, 255, 0)

    contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    #l=image.copy()

    #blobs_dog[:, 2] = blobs_dog[:, 2] *(2)**(0.5)
    mask2 = np.zeros(image.shape[:2], dtype= np.uint8)
    cv.drawContours(mask2, contours, -1, (255,255,255), -1)
    #for pt in zip(blobs_dog): 
    #    #print(pt)
    #    mask2[pt[1]:pt[1]+round(pt[3]/2), pt[0]:pt[0]+round(pt[3]/2)] = 255
    #    # a rectangle is drawn on the mask, which marks where the points are 

    image_rgb=image.copy()
    image_blocked = cv.bitwise_and(image_rgb, image_rgb, mask=mask2)
    #plt.imshow(image_blocked)
    return image_blocked

def imgregfun(imagebef, imageafter):
#### A function for image registration, stolen of the internet but I can't remember where from
###Inputs: imagebef- the before image
##########imageafter- the after image
###outputs: transimaf- the translated after image
    # Open the image files.
    img1_color = imageafter  # Image to be aligned.
    img2_color = imagebef  # Reference image.

    # Convert to grayscale.
    img1 = cv.cvtColor(img1_color, cv.COLOR_BGR2GRAY)
    img2 = cv.cvtColor(img2_color, cv.COLOR_BGR2GRAY)
    height, width = img2.shape

    # Create ORB detector with 5000 features.
    orb_detector = cv.ORB_create(5000)

    # Find keypoints and descriptors.
    # The first arg is the image, second arg is the mask
    #  (which is not reqiured in this case).
    kp1, d1 = orb_detector.detectAndCompute(img1, None)
    kp2, d2 = orb_detector.detectAndCompute(img2, None)

    # Match features between the two images.
    # We create a Brute Force matcher with
    # Hamming distance as measurement mode.
    matcher = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)

    # Match the two sets of descriptors.
    matches = matcher.match(d1, d2)

    # Sort matches on the basis of their Hamming distance.
    matches.sort(key=lambda x: x.distance)

    # Take the top 90 % matches forward.
    matches = matches[:np.int(len(matches) * 90)]
    no_of_matches = len(matches)

    # Define empty matrices of shape no_of_matches * 2.
    p1 = np.zeros((no_of_matches, 2))
    p2 = np.zeros((no_of_matches, 2))

    for i in range(len(matches)):
        p1[i, :] = kp1[matches[i].queryIdx].pt
        p2[i, :] = kp2[matches[i].trainIdx].pt

    # Find the homography matrix.
    homography, mask = cv.findHomography(p1, p2, cv.RANSAC)

    # Use this matrix to transform the
    # colored image wrt the reference image.
    transformed_img = cv.warpPerspective(img1_color, homography,
                                          (width, height))
    transimaf=transformed_img
    return transimaf
def imageprocessingfunction(beforefol,afterfol):
###A function for getting all the CR2 files within the before and after folders, then reading them
### and saving them on the disk as virtual images 
##########################################################################
###Inputs: beforefol: selected before folder
##########afterfol: selected after folder 
###Outputs: imaf: the images in the after folder as an array
###########imbef: the images in the before folder as an array 
###########beforeimfile: the list of before image files
###########afterimfile: the list of after image files
    # Get file list
    beforeimfile=glob.glob(beforefol+"\\"+"*.CR2")
    afterimfile=glob.glob(afterfol+"\\"+"*.CR2")
    #print(afterimfile)

    #Exifdata is just there in case you need to edit the images in a fancy way.
    imaf,labaf,imbef,labef=[],[],[],[]
    for impath in afterimfile:
        image,exifdata=   convertfilefun(impath)
        imaf.append(np.dstack((image)))
        labaf.append(exifdata)
    for impath in beforeimfile:
        image,exifdata= convertfilefun(impath)
        imbef.append(np.dstack((image)))
        labef.append(exifdata)
    return imaf,imbef,beforeimfile,afterimfile

def convertfilefun(path):
## a function which converts CR2 images to TIFF images the computer can actually read
## input: path- path to raw image
## output : an image that is readable using cv2
    with rawpy.imread(path) as raw:
        #Can fiddle with camera settings but I wouldn't reccoment it
        rgb = raw.postprocess(use_camera_wb=True,
                              no_auto_bright=True,
                              gamma=(2.222, 4.5),
                              chromatic_aberration=(1, 1))
        #cv2.imwrite(path + '.tiff',rgb)
        # extract EXIF data to save as metadata
        metdat = Image.open(path)
        exifdata = metdat.getexif()
        image = rgb
        image = rgb.reshape(
            (1, image.shape[0], image.shape[1], image.shape[2]))
        return image, exifdata
        #plt.imsave(path + '.png',rgb)
        #g=print(path + '.png')
        #return g
        
def saveexcelfun(dimercoord,corecoord,savepath):
#Save the coordinates of everything to an excel spreadsheet. Yeah I know it's ugly. 
#input: dimercoord- the dimer coordinates
########corecoord- the core coordinates 
########savepath- the folder location where the files will be saved 
    columns=['Before Dimers xval']
    saveexcel=savepath+"."+"xlsx"
    beforedimercentres = pd.DataFrame({'Before Dimers xval':dimercoord})
    try:
        aftercorecentres = pd.DataFrame({'After core xval': corecoord})
    except:
        aftercorecentres=pd.DataFrame({'After core xval': np.array([0]), 
                                'After core yval': np.array([0])})

   
    writer = pd.ExcelWriter(saveexcel,engine='xlsxwriter')
    workbook=writer.book
    worksheet=workbook.add_worksheet('DimersPicked')
    writer.sheets['DimersPicked'] = worksheet
    worksheet2=workbook.add_worksheet('CoresPicked')
    writer.sheets['CoresPicked'] = worksheet2


    beforedimercentres.to_excel(writer,sheet_name='DimersPicked',startrow=1 , startcol=0)
    #worksheet.write_string(beforedimercentres.shape[0] + 4, 0, beforedimercentres.name)

    aftercorecentres.to_excel(writer,sheet_name='CoresPicked',startrow=1, startcol=3)
    

    writer.save()
def savetextfilefun(data,savepath,datastring):
##### A function which saves an array to a text file. Is a little buggy in that sometimes there's weird spaces. 
##### reccomend the excel save functions instead. Python struggles to re-read these text tiles

    savetextstring=savepath+datastring+".txt"
    file = open(savetextstring,"w")
    for dataentry in data:
        arr_of_strings = np.array2string(dataentry)
        file.write(arr_of_strings) 
    file.close() 
    

def load_images_from_folder(folder):
## A function which reads in images and adds them to a list of images.
## input: folder- a string which points to the folder location
## output: images, a list of images within the folder
## note !! The images wil be read in with open cv, and will be in BGR format and will look strange unless converted
    images = []
    for filename in os.listdir(folder):
        img = cv.imread(os.path.join(folder,filename))        
        if img is not None:
            images.append(img)
    return images
def load_images_from_foldercv(folder):
## A function which reads in images and adds them to a list of images. This function also converts them to RGB
## input: folder- a string which points to the folder location
## output: images, a list of images within the folder

    images = []
    for filename in os.listdir(folder):
        img = cv.imread(os.path.join(folder,filename))
       
        if img is not None:
            img= cv.cvtColor(img, cv.COLOR_BGR2RGB)
            images.append(img)
            
    return images
def load_images_from_folderhsv(folder):
## A function which reads in images and adds them to a list of images. This function also converts them to HSV
## input: folder- a string which points to the folder location
## output: images, a list of images within the folder
    images = []
    for filename in os.listdir(folder):
        img = cv.imread(os.path.join(folder,filename))
        img= cv.cvtColor(img,  cv.COLOR_BGR2HSV)
        if img is not None:
            images.append(img)
    return images

########################################
#Template matching functions

def templatematchingalgorithm(img_hsv,template, method,threshold):
# defines the template matching algorithm and finds the minimum locations
#inputs: img_rgb- the image to be matched in HSV format
#########template- the template to be matched in HSV format
#########method- the method of determining the minim. cv.TM_SQDIFF_NORMED is good for colour
##### extras found here: https://docs.opencv.org/master/df/dfb/group__imgproc__object.html#ga3a7850640f1fe1f58fe91a2d7583695d
#########threshold: the threshold where the minimum is defined. Variable. May want to do something with min_val and max
#outputs: locations_of_minimum- a really big array that needs to be zipped. 
#Note: this works on colour (3 d) images but may want to change to just hue
    mat_of_matching_results=cv.matchTemplate(img_hsv,template,method) 
    #This is to get some details about the minimum but isn't actually used
    (min_val, max_val, _, max_loc) = cv.minMaxLoc(mat_of_matching_results)
    #print(min_val)
    #print(max_val)
    locations_of_minimum = np.where( mat_of_matching_results <= ((max_val-min_val)/threshold)+min_val)
    #print(locations_of_minimum)
    return locations_of_minimum

def checkfordoublecounting(img_hsv,locations_of_min):
## a function which checks for multiple counting of the same point by drawing a box around a point and checking
## that there's no pre-drawn box in the sampling area
# input: img_hsv: a large hsv image that has been template matched
######## locations_of_min: the locations selected as minima by the template matching
#output: particle_count: the locations of the particles selected in a list format
    particle_count=[]
    # a copy of the image is used so there aren't random boxes drawn over the image
    scanimage=img_hsv.copy()
    mask = np.zeros(scanimage.shape[:2], dtype=np.uint8)
    w=32
    h=32
    
    #initialising the mask. The type must be the same
    #print("in the double count locations are",locations_of_min)
    for pt in zip(*locations_of_min[::-1]):           
        try: 
            
            #if there is no predrawn box around the point sampled, add the point to a list of coordinates
            if mask[pt[1] + h, pt[0] + w] != 255:
                
                mask[pt[1]:pt[1]+h, pt[0]:pt[0]+w] = 255
                particle_count.append(pt)
                cv.rectangle(mask, pt, (pt[0] + w, pt[1] + h), (0,255,0), 1)
        except:
            z=1  
   # print("The attained particle count is ", particle_count)
    return particle_count
        
    
    
def imagetempmatch(imgpath,img_rgb,threshold):
## performs the template matching function on each template image found in imgpath
## input: imgpath: a string pointing to the folder the template images are contained in
##########img_rgb: the image to be matched, in rgb format 
##########threshold: the threshold at which the minimum is accepted. The minimum value is the value where the 
#####################template matching function thinks that the image is matched. Note, this may be a maximum 
####################for other methods
    #loads the template images in as HSV
    images=load_images_from_folderhsv(imgpath)
    #Creates a mask which has the shape of the image to be matched. The dtype is important or error will occur. 
    # This mask is to test whether the template matching has counted the same point multiple times
    mask = np.zeros(img_rgb.shape, dtype=np.uint8)
    res=[[]];
    for template in images:
        #for each template, the width and height is taken
        
        w, h = template.shape[0:2]
        #big image converted to hsv format
        img_hsv= cv.cvtColor(img_rgb,  cv.COLOR_RGB2HSV)
        #the location minima identified with the template matching algorithm
        locations_of_min=templatematchingalgorithm(img_hsv,template,cv.TM_SQDIFF_NORMED,threshold)
        #print("locations of min are", locations_of_min)
        # the locations are checked for multiple counting of the same point. 
        particle_count=checkfordoublecounting(img_hsv,locations_of_min)
        #print(particle_count)
    return particle_count

def blockimage(image_rgb, points,w,h):
### a function which returns an image where the selected points are blocked out with a white rectangle. 
### input: image_rgb- the large image in RGB format, to be blocked
########## points- the points to draw rectangles on
###########w,h- the width and height (in pixels) respectively of the rectangle to be drawn
    for pt in points:
        image_rgb = cv.rectangle(image_rgb,pt, (pt[0] + w, pt[1] + h) , (255,255,255), -1)
    return image_rgb

def blockimagemask(image_rgb,points,w,h):
### a function which returns an image where everything but the selected points is blocked out  
### input: image_rgb- the large image in RGB format, to be blocked
########## points- the points to draw rectangles on
###########w,h- the width and height (in pixels) respectively of the rectangle to be drawn
    # the mask is generated, the dtype is important
    h=32
    w=32
    mask2 = np.zeros(image_rgb.shape[:2], dtype= np.uint8)
    for pt in points: 
        #print(pt)
        mask2 = cv.rectangle(mask2,pt, (pt[0] + w, pt[1] + h) , (255,255,255), -1)
        
        # a rectangle is drawn on the mask, which marks where the points are 
    #invmask=255-mask2
    #This is an inbuilt cv function which clips the image around the mask. 
    image_blocked = cv.bitwise_and(image_rgb, image_rgb, mask=mask2)
    #plt.imshow(img_rgb)
    return image_blocked

def performtemplatematching(image_rgb,imgpath,bok,threshold):
## a function which performs template matching on the images and blocks them depending on whether
## we want to keep the particles or block them out for more accuracy 
#input: image_rgb: the big image in rgb format where we are looking for matches
########imgpath: the path to the template images, as a string
#########bok: block or keep. 0 is for keeping, 1 is for blocking with a rectange
#########threshold: the threshold for the minimum values. Variable. 
    points=imagetempmatch(imgpath,image_rgb,threshold)
    w=32
    h=32
    print(points)
    if bok==1:
        print(points)
        correctedimg=blockimage(image_rgb,points,w,h)
    else:
        print(points)
        correctedimg=blockimagemask(image_rgb,points,w,h)
    return correctedimg,points
def performsaveimage(image,path):
## saves image using pillow, which is a lot faster than matplot lib. 
    img_rgb_corr=image
    im_pil = Image.fromarray(img_rgb_corr)
    im_pil.save(path, compress_level=1)

def savetotrainingfol(foldername,image,points):
## saves the cropped images to a folder, for use in machine learning. uses a 16 pixel box.
#input: foldername: name of the folder where you want the images stored
#######image: an image in rgb format which you want to cut up
########points: the coordinates of the particles which you have selected. 
    boxwid=round(16/2)
    for j,pt in enumerate(points):
        savepath=os.path.join(foldername, str(j)+"registeredimg" + "." + "png")
        lilimage=image[pt[1]:pt[1]+30, pt[0]:pt[0]+30]
        performsaveimage(lilimage,savepath)
def blockoutunwantedparticles(analpath,sat_img,path,threshold):
## a function which blocks out that particles which are interfering with analysis i.e. clumps and clusters
#inputs: analpath: the path to the analysis folder
######## sat_img: the before target image in rgb format
#output: correctedimgcore, the corrected image after accounting for cores and clumps
# These images are hardcoded into a folder, so the code has some dependencies. But any exmaples of the correct size will do
    #The threshold of 0.25 seems to be highly variable
    correctedimg,_= performtemplatematching(sat_img, path,1,threshold)
    return correctedimg

def unwantedparticleblocking(analpath,sat_img):
    #clusterpath=  r"D:\OneDrive - UNSW\Image_Analysis\typesofParticles\NewAItraining\Cluster"
    #threshold= 0.25
    #correctedimgcluster=blockoutunwantedparticles(analpath,sat_img,clusterpath,threshold)
    
    clumppath= r"D:\OneDrive - UNSW\Image_Analysis\typesofParticles\NewAItraining\newmicroscope\clump"
    threshold=2
    savepath=os.path.join(analpath, "clumpcorrectedimg" + "." + "png")
    correctedimgclump=blockoutunwantedparticles(analpath,sat_img,clumppath,threshold)
    performsaveimage(correctedimgclump,savepath)
    
    corepath= r"D:\OneDrive - UNSW\Image_Analysis\typesofParticles\NewAItraining\newmicroscope\core"
    threshold=2
    correctedimgcore=blockoutunwantedparticles(analpath,correctedimgclump,corepath,threshold)
    savepath=os.path.join(analpath, "corecorrectedimg" + "." + "png")
    performsaveimage(correctedimgcore,savepath)
    
    return correctedimgcore
    
def keepmatchedparticles(path,image,threshold):
## a function which keeps the wanted particles and blocks out the rest. 
#inputs: analpath: the path to the analysis folder
######## sat_img: the before target image in rgb format
#########path: path to the images to analyse
    correctedimgdimer,pointsdimer= performtemplatematching(image, path, 0, threshold)
    
    return correctedimgdimer,pointsdimer
                                                              

def performkeepmatchedparticles(analpath,image):
## a function which keeps the wanted particles and blocks out the rest. 
#inputs: analpath: the path to the analysis folder
######## sat_img: the before target image in rgb format
#########path: path to the images to analyse
#outputs: the selected image and the dimer points selected
    dimerpath= r"D:\OneDrive - UNSW\Image_Analysis\typesofParticles\NewAItraining\newmicroscope\dimer"
    threshold=6
    correctedimgdimer,pointsdimer=keepmatchedparticles(dimerpath,image,threshold)
    savepath=os.path.join(analpath, "pickeddimersimg" + "." + "png")
    performsaveimage(correctedimgdimer,savepath)
    return correctedimgdimer,pointsdimer

def getaveragevalues(image,points,w,h,analpath):
# a function to get the average values of the image in rgb format
#inputs: image: the rgb image
#########points: the selected dimer values
#########w,h- the width and height in pixels
#########analpath: the path to the analysis folder
#outputs: an array of the average colour of the red divided by the green vector

    
    average_colour_rDivg=[]
    for i,pt in enumerate(points):           
        try: 
            crop_im=image[pt[1]:pt[1]+h, pt[0]:pt[0]+w]
            dividedval=np.divide(crop_im[0].reshape((1,-1)),crop_im[1].reshape((1,-1)))
            average_colour_rDivg.append(np.mean(dividedval))            

        except:
            z=1;    
    
    return np.array(average_colour_rDivg)

def getaveragevalueshsv(image,points,w,h,analpath):
# a function to get the average values of the image in rgb format
#inputs: image: the hsv image
#########points: the selected dimer values
#########w,h- the width and height in pixels
#########analpath: the path to the analysis folder
#outputs: an array of the average colour of the red divided by the green vector

    average_colour_rDivg=[]
    #converts the image to hsv
    image = cv.cvtColor(image,  cv.COLOR_RGB2HSV)
    for i,pt in enumerate(points):           
        try: 
            crop_im=image[pt[1]:pt[1]+h, pt[0]:pt[0]+w]
            dividedval=image[:,:,0].reshape((1,-1))
            #print(dividedval)
            average_colour_rDivg.append(np.mean(image[0]))      
        except:
            z=1;    
    
    return np.array(average_colour_rDivg)

def averagehistogramshift(correctedimgdimer,pointsdimer,correctedimgtarget,threshold):
# a function which uses the average shift of the particle colour (either hue or rgb depending ) to 
#select the particles which have shifted in the after image 
#inputs: correctedimgdimer: the image with only dimers selected
#########pointsdimer: the locations of the dimers in the image
#########correctedimgtarget: the registered image after target added wherein the coordinates of the selected
# dimers have been used to clip it
#########threshold: the values which the shifted average value must be above or below. was 0.2 
#output: loc- the shifted selected particles. 
    w=32
    h=32
    avdim=getaveragevalueshsv(correctedimgdimer,pointsdimer,w,h, analpath)
    avcore=getaveragevalueshsv(correctedimgtarget,pointsdimer,w,h, analpath)
    minus= np.array(avdim)-np.array(avcore)
    loc=np.array(pointsdimer)[minus < threshold]
    targetpicked=blockimagemask(correctedimgtarget,loc,w,h)
    savepath=os.path.join(analpath, "tarmaskedimg" + "." + "png")
    performsaveimage(targetpicked,savepath)
    return loc
    



        
        
def performtemplatetrainingonimages(transtarimg,analpath,sat_img):

    #blocks unwated particles
    correctedimgcore=unwantedparticleblocking(analpath,sat_img)
    
    #selects dimers
    correctedimgdimer,pointsdimer=performkeepmatchedparticles(analpath,correctedimgcore)
    

    #uses those points to select the same points in the target image 
    w=32
    h=32
    correctedimgtarget=blockimagemask(transtarimg,pointsdimer,w,h)
    savepath=os.path.join(analpath, "targetblocked" + "." + "png")
    performsaveimage(correctedimgtarget,savepath)
    
    #scans the before and after dimers to determine if the particles shift in hue
    threshold=-0.1
    selected_target_locations=averagehistogramshift(correctedimgdimer,pointsdimer,correctedimgtarget,threshold)
     
    
    
    #comment out if don't want
    #savetotrainingfol(r"D:\OneDrive - UNSW\Image_Analysis\typesofParticles\NewAItraining\newmicroscope\unsorted",sat_img,pointsdimer)
    #savetotrainingfol(r"D:\OneDrive - UNSW\Image_Analysis\typesofParticles\NewAItraining\newmicroscope\unsorted",transtarimg,pointsdimer)
    
    return selected_target_locations,pointsdimer


######### Extra functions

def getindividualfoldersandsuch(bigfol):
# a function which gets the folders underneath the big folder. Must be the format: bigfol>slide>sample>satellite | target> images
    topfolders=listdirs(bigfol)
    subfolders=[0]
    for nextfolders in topfolders:
        subfolders= subfolders+ listdirs(nextfolders)
    subfolders.pop(0)
    #print(subfolders)
    subfolders=np.array(subfolders).reshape(-1,1)
    #print(subfolders)
    return subfolders
def searchforsatandtargetfolders(folder):
#finds the satellite and targetfolders using regexp
    subfolders2 = folder.tolist()
    subfolders3=str(subfolders2).replace('[','').replace(']','').replace('\\\\','\\')

    listexpfolders=listdirs(subfolders3[1:len(subfolders3)-1])
    beforefol = [x for x in listexpfolders if re.search("satellite",x)]
    beforefol=beforefol[0]
    afterfol= [x for x in listexpfolders if re.search("target",x)]
    afterfol=afterfol[0]
    
    return beforefol,afterfol

def createanalysisfolder(beforefol):
#creates an analysis folder in the address above where the satellite and target folders are located.
    oneuppath=os.path.dirname(beforefol)
    analysisfolderpath=oneuppath+"\\"+"Analysisfolder"
    try:
        os.mkdir(analysisfolderpath)
    except:
        print("folder already exists")
    print(analysisfolderpath)
    print("processing images")
    return analysisfolderpath,oneuppath

def createimagesubfolderforsaving(pathtomatch,analysisfolderpath):
# creates a folder with the name of the image in the analysis folder 
    pathtomatch=beforeimfile[j]
    matchingsearch=re.search("IMG_.*.CR2",pathtomatch)
    savefilespath=analysisfolderpath+"\\"+matchingsearch.group()+"\\"
    try: 
        os.mkdir(savefilespath)
    except:
        print("folder already exists")
    return savefilespath
def addstringwithtime(savefilespath):
# adds the current date and time so there's no saving over the top of different analysis
    now = datetime.now()
    dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
    savepath=savefilespath+dt_string
    return savepath
def makeanalysisfolder(savefilespath):
# makes a directory in the folder which matches the image name which says 'analysis'
    analpath=os.path.join(savefilespath,"Analysisimages")
                        #saves the registered image


    try: 
        os.mkdir(analpath)
    except:
        print("folder already exists")
    
    return analpath


####################
#template matching main 

In [8]:
bigfol=r"D:\OneDrive - UNSW\Image_Analysis\NewMicroscope\OneDrive_2021-06-28\Time series experiment for analysis\Test"
subfolders=getindividualfoldersandsuch(bigfol)

for folder in subfolders:
    #automatically find the before and after folder 
    beforefol,afterfol= searchforsatandtargetfolders(folder)
    
    #Create the relevant folder for saving,as well as the path one folder up 
    analysisfolderpath,oneuppath=createanalysisfolder(beforefol)
    
    #loadsall the images in 

    imaf,imbef,beforeimfile,afterimfile=imageprocessingfunction(beforefol,afterfol)
    print("processed and converted images")
    
    
    response=[0]*len(beforeimfile)
    dimercount=[0]*len(beforeimfile)
    targetcount=[0]*len(beforeimfile)
    #t = tqdm(total=len(imbef))
    for j,image in enumerate(imbef):
        with tqdm(total=len(imbef), file=sys.stdout) as pbar:

            #pbar.write('processed: %d' % (j+1))
            pbar.update(1)
            sleep(0.1)

        pathtomatch=beforeimfile[j]
        savefilespath= createimagesubfolderforsaving(pathtomatch,analysisfolderpath)
        
        #add on the currentdate and time 
        savepath=addstringwithtime(savefilespath)
        
        
        print("calculating image "+str(j+1)+" of "+ str(len(imbef)))
        
        #make an analysis folder which says analysis in the image folder just created 
        analpath=makeanalysisfolder(savefilespath)
        
        #spot matches the before and after images 
        transimaf=imgregfun(image, imaf[j])
        #saves the redistered image
        saveregisteredimage=os.path.join(analpath, "registeredimg" + "." + "png")
        performsaveimage(transimaf,saveregisteredimage)
        
        #uncomment if you want a crop box    
        #height, width = image.shape[:2]
        #boxwid=round(1500/2)
        #centreimagebefore=image[round(height/2)-boxwid:round(height/2)+boxwid,round(width/2)-boxwid:round(width/2)+boxwid]
        #centreimageafter=transimaf[round(height/2)-boxwid:round(height/2)+boxwid,round(width/2)-boxwid:round(width/2)+boxwid]
        centreimagebefore=makeclippingmask(image)
        centreimageafter=makeclippingmask(transimaf)
        
        
        #perform the template matching and see which particles shift
        targetpicked,pointsdimer=performtemplatetrainingonimages(centreimageafter,analpath,centreimagebefore)
        
        #saves the to an excel spreadsheet and saves the results to a list to get the overall results at the end 
        #if I feel bothered I can add something in that uses the already analysed images but meh 
        dimercount[j]=len(pointsdimer)
        targetcount[j]=len(targetpicked)
        try: 
            response[j]=(len(targetpicked)/len(pointsdimer))*100
        except:
            response[j]=0
        
        saveexcelfun(pointsdimer,targetpicked,savepath)
        
        #save important coordinates to a text file for later analysis
        #Probably can delete this don't know what it's used for anymore 
        #savetextfilefun(np.array(pointsdimer),savepath,"dimerbefore")
        #savetextfilefun(np.array(targetpicked),savepath,"selectedResponse")

    print("Finished analysis woohoo")
    #t.close()
    #Find the folders which contain the analysis and read the text files
    #Find the image folders
    searchfolder=oneuppath
    #insearchfolder=os.listdir(searchfolder)
    savedir= searchfolder+'\\'+"Analysisfolder"
    imagefilefolders=os.listdir(savedir)
    #Write excel worsheet in the analysis folder
    responseforexcel = pd.DataFrame({'dimers_picked':np.array(dimercount),'target_picked':np.array(targetcount),'response': np.array(response)})
    responsepath=savedir+'\\'+"responseforallimages.xlsx"
    responseforexcel.to_excel(responsepath) 

folder already exists
D:\OneDrive - UNSW\Image_Analysis\NewMicroscope\OneDrive_2021-06-28\Time series experiment for analysis\Test\slide1\initial\Analysisfolder
processing images
processed and converted images


  0%|          | 0/1 [00:00<?, ?it/s]

folder already exists
calculating image 1 of 1
folder already exists
[(2858, 396), (2859, 396), (2860, 396), (2861, 396), (2862, 396), (2855, 397), (2856, 397), (2857, 397), (2858, 397), (2859, 397), (2860, 397), (2861, 397), (2862, 397), (2863, 397), (2864, 397), (2865, 397), (2866, 397), (2867, 397), (2854, 398), (2855, 398), (2856, 398), (2857, 398), (2858, 398), (2859, 398), (2860, 398), (2861, 398), (2862, 398), (2863, 398), (2864, 398), (2865, 398), (2866, 398), (2867, 398), (2868, 398), (2869, 398), (2870, 398), (2871, 398), (2872, 398), (2853, 399), (2854, 399), (2855, 399), (2856, 399), (2857, 399), (2858, 399), (2859, 399), (2860, 399), (2861, 399), (2862, 399), (2863, 399), (2864, 399), (2865, 399), (2866, 399), (2867, 399), (2868, 399), (2869, 399), (2870, 399), (2871, 399), (2872, 399), (2873, 399), (2874, 399), (2852, 400), (2853, 400), (2854, 400), (2855, 400), (2856, 400), (2857, 400), (2858, 400), (2859, 400), (2860, 400), (2861, 400), (2862, 400), (2863, 400), (2864, 

[(2856, 400), (2857, 400), (2858, 400), (2859, 400), (2860, 400), (2861, 400), (2862, 400), (2863, 400), (2864, 400), (2865, 400), (2866, 400), (2867, 400), (2868, 400), (2869, 400), (2870, 400), (2855, 401), (2856, 401), (2857, 401), (2858, 401), (2859, 401), (2860, 401), (2861, 401), (2862, 401), (2863, 401), (2864, 401), (2865, 401), (2866, 401), (2867, 401), (2868, 401), (2869, 401), (2870, 401), (2871, 401), (2872, 401), (2873, 401), (2854, 402), (2855, 402), (2856, 402), (2857, 402), (2858, 402), (2859, 402), (2860, 402), (2861, 402), (2862, 402), (2863, 402), (2864, 402), (2865, 402), (2866, 402), (2867, 402), (2868, 402), (2869, 402), (2870, 402), (2871, 402), (2872, 402), (2873, 402), (2874, 402), (2875, 402), (2854, 403), (2855, 403), (2856, 403), (2857, 403), (2858, 403), (2859, 403), (2860, 403), (2861, 403), (2862, 403), (2863, 403), (2864, 403), (2865, 403), (2866, 403), (2867, 403), (2868, 403), (2869, 403), (2870, 403), (2871, 403), (2872, 403), (2873, 403), (2874, 403)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[(2196, 682), (2197, 682), (2196, 683), (2197, 683), (2195, 684), (2196, 684), (2194, 685), (2195, 685), (2196, 685), (2193, 686), (2194, 686), (2195, 686), (2191, 687), (2192, 687), (2193, 687), (2194, 687), (2195, 687), (2190, 688), (2191, 688), (2192, 688), (2193, 688), (2191, 689), (2192, 689), (2190, 690), (2191, 690), (2191, 691), (1322, 2098), (1323, 2098), (1324, 2098), (1325, 2098), (1326, 2098), (1327, 2098), (1320, 2099), (1321, 2099), (1322, 2099), (1323, 2099), (1324, 2099), (1325, 2099), (1326, 2099), (1327, 2099), (1328, 2099), (1319, 2100), (1320, 2100), (1321, 2100), (1322, 2100), (1323, 2100), (1324, 2100), (1325, 2100), (1326, 2100), (1327, 2100), (1328, 2100), (1329, 2100), (1318, 2101), (1319, 2101), (1320, 2101), (1321, 2101), (1322, 2101), (1323, 2101), (1324, 2101), (1325, 2101), (1326, 2101), (1327, 2101), (1328, 2101), (1329, 2101), (1330, 2101), (1317, 2102), (1318, 2102), (1319, 2102), (1320, 2102), (1321, 2102), (1322, 2102), (1323, 2102), (1324, 2102), (13

[(1322, 2099), (1323, 2099), (1324, 2099), (1325, 2099), (1326, 2099), (1327, 2099), (1321, 2100), (1322, 2100), (1323, 2100), (1324, 2100), (1325, 2100), (1326, 2100), (1327, 2100), (1328, 2100), (1321, 2101), (1322, 2101), (1323, 2101), (1324, 2101), (1325, 2101), (1326, 2101), (1327, 2101), (1328, 2101), (1329, 2101), (1319, 2102), (1320, 2102), (1321, 2102), (1322, 2102), (1323, 2102), (1324, 2102), (1325, 2102), (1326, 2102), (1327, 2102), (1328, 2102), (1329, 2102), (1330, 2102), (1319, 2103), (1320, 2103), (1321, 2103), (1322, 2103), (1323, 2103), (1324, 2103), (1325, 2103), (1326, 2103), (1327, 2103), (1328, 2103), (1329, 2103), (1330, 2103), (1331, 2103), (1319, 2104), (1320, 2104), (1321, 2104), (1322, 2104), (1323, 2104), (1324, 2104), (1325, 2104), (1326, 2104), (1327, 2104), (1328, 2104), (1329, 2104), (1330, 2104), (1331, 2104), (1332, 2104), (1319, 2105), (1320, 2105), (1321, 2105), (1322, 2105), (1323, 2105), (1324, 2105), (1325, 2105), (1326, 2105), (1327, 2105), (1328

[(1322, 2098), (1323, 2098), (1324, 2098), (1321, 2099), (1322, 2099), (1323, 2099), (1324, 2099), (1325, 2099), (1326, 2099), (1320, 2100), (1321, 2100), (1322, 2100), (1323, 2100), (1324, 2100), (1325, 2100), (1326, 2100), (1319, 2101), (1320, 2101), (1321, 2101), (1322, 2101), (1323, 2101), (1324, 2101), (1325, 2101), (1326, 2101), (1327, 2101), (1319, 2102), (1320, 2102), (1321, 2102), (1322, 2102), (1323, 2102), (1324, 2102), (1325, 2102), (1326, 2102), (1327, 2102), (1319, 2103), (1320, 2103), (1321, 2103), (1322, 2103), (1323, 2103), (1324, 2103), (1325, 2103), (1326, 2103), (1327, 2103), (1328, 2103), (1319, 2104), (1320, 2104), (1321, 2104), (1322, 2104), (1323, 2104), (1324, 2104), (1325, 2104), (1326, 2104), (1327, 2104), (1328, 2104), (1318, 2105), (1319, 2105), (1320, 2105), (1321, 2105), (1322, 2105), (1323, 2105), (1324, 2105), (1325, 2105), (1326, 2105), (1327, 2105), (1328, 2105), (1319, 2106), (1320, 2106), (1321, 2106), (1322, 2106), (1323, 2106), (1324, 2106), (1325

[(3364, 2879), (3365, 2879), (3366, 2879), (3365, 2880), (290, 2921), (291, 2921), (290, 2922), (291, 2922), (2086, 2971), (2087, 2971), (2085, 2972), (2086, 2972), (2087, 2972), (2088, 2972), (2084, 2973), (2085, 2973), (2086, 2973), (2087, 2973), (2088, 2973), (2089, 2973), (2084, 2974), (2085, 2974), (2086, 2974), (2087, 2974), (2088, 2974), (2089, 2974), (2085, 2975), (2086, 2975), (2087, 2975)]
[(1512, 2780), (1511, 2781), (1512, 2781), (1511, 2782), (1512, 2782), (1511, 2783), (165, 2846), (166, 2846)]
[(258, 1820), (258, 1821), (1758, 2641), (1758, 2642), (2706, 2747), (2707, 2747), (2707, 2748), (1508, 2781), (1509, 2781), (1510, 2781), (1507, 2782), (1508, 2782), (1509, 2782), (1510, 2782), (1507, 2783), (1508, 2783), (1509, 2783), (1510, 2783), (1508, 2784), (1509, 2784), (3361, 2891), (3362, 2891), (3360, 2892), (3361, 2892), (3362, 2892), (286, 2931), (287, 2931), (286, 2932), (287, 2932), (288, 2932), (286, 2933), (287, 2933), (286, 2934), (287, 2934), (2297, 3119)]
[(258,

In [4]:

commentout if you want to test

imaf,imbef,beforeimfile,afterimfile=imageprocessingfunction(beforefol,afterfol)

j=0
image=imbef[0]
pathtomatch=beforeimfile[j]
savefilespath= createimagesubfolderforsaving(pathtomatch,analysisfolderpath)

#add on the currentdate and time 
savepath=addstringwithtime(savefilespath)


print("calculating image "+str(j+1)+" of "+ str(len(imbef)))

#make an analysis folder which says analysis in the image folder just created 
analpath=makeanalysisfolder(savefilespath)

#spot matches the before and after images 
transimaf=imgregfun(image, imaf[j])
#saves the redistered image
saveregisteredimage=os.path.join(analpath, "registeredimg" + "." + "png")
performsaveimage(transimaf,saveregisteredimage)

##uncomment if you want a crop box    
##height, width = image.shape[:2]
##boxwid=round(1500/2)
##centreimagebefore=image[round(height/2)-boxwid:round(height/2)+boxwid,round(width/2)-boxwid:round(width/2)+boxwid]
###centreimageafter=transimaf[round(height/2)-boxwid:round(height/2)+boxwid,round(width/2)-boxwid:round(width/2)+boxwid]
#centreimagebefore=image
#centreimageafter=transimaf
#compare the two images to see if registered properly
# from https://www.pyimagesearch.com/2017/06/19/image-difference-with-opencv-and-python/
#from skimage.measure import compare_ssim
# convert the images to grayscale
#imageclippedbefore=makeclippingmask(imbef[0])
#diffim=cv.imread(r"D:\OneDrive - UNSW\Image_Analysis\IMAGE ANALYSIS NO BLUR\slide8-230920-cal\1- 1nM\satellite\IMG_0005s1.CR2.png")
#diffim=transimaf
#imageclippedafter=makeclippingmask(diffim)
#grayA = cv.cvtColor(imageclippedbefore, cv.COLOR_RGB2GRAY)

#grayB = cv.cvtColor(imageclippedafter, cv.COLOR_RGB2GRAY)

#(score, diff) = compare_ssim(grayA, grayB, full=True)
#diff = (diff * 255).astype("uint8")
#print("SSIM: {}".format(score))

# threshold the difference image, followed by finding contours to
# obtain the regions of the two input images that differ
#threshdiff = cv.threshold(diff, 0, 255,cv.THRESH_BINARY_INV | cv.THRESH_OTSU)[1]
#cnts = cv.findContours(diff, cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
#cnts = imutils.grab_contours(cnts)


#imb=image.copy()
#ima=diffim.copy()
# loop over the contours
#for c in cnts:
    # compute the bounding box of the contour and then draw the
    # bounding box on both input images to represent where the two
    # images differ
#    (x, y, w, h) = cv.boundingRect(c)
#    cv.rectangle(ima, (x, y), (x + w, y + h), (0, 0, 255), 2)
#    cv.rectangle(imb, (x, y), (x + w, y + h), (0, 0, 255), 2)
# show the output images

#plt.imshow(imageclippedafter)
#print(len(cnts))
#plt.figure(figsize = (10,10))
#plt.imshow(ima)
#plt.imshow(imb, alpha=0.5)
#plt.imshow(threshdiff)
#plots a figure which shows where a and b differ
#correctedimg,_= performtemplatematching(sat_img, path,1,threshold)




SyntaxError: invalid syntax (<ipython-input-4-d0d666fcdeb1>, line 1)

In [ ]:
clumppath= r"D:\OneDrive - UNSW\Image_Analysis\typesofParticles\NewAItraining\clump"
threshold=0.25
savepath=os.path.join(analpath, "clumpcorrectedimg" + "." + "png")
correctedimgclump=blockoutunwantedparticles(analpath,imbef[0].copy(),clumppath,threshold)
plt.imshow(correctedimgclump)

In [ ]:
comment is here
clusterpath=  r"D:\OneDrive - UNSW\Image_Analysis\typesofParticles\NewAItraining\Cluster"
threshold= 0.25
correctedimgcluster=blockoutunwantedparticles(analpath,sat_img,clusterpath,threshold)

clumppath= r"D:\OneDrive - UNSW\Image_Analysis\typesofParticles\NewAItraining\clump"
threshold=0.25
savepath=os.path.join(analpath, "clumpcorrectedimg" + "." + "png")
correctedimgclump=blockoutunwantedparticles(analpath,correctedimgcluster,clumppath,threshold)
performsaveimage(correctedimgclump,savepath)

corepath= r"D:\OneDrive - UNSW\Image_Analysis\typesofParticles\NewAItraining\newmicroscope\core"
threshold=0.05
correctedimgcore=blockoutunwantedparticles(analpath,correctedimgclump,corepath,threshold)
savepath=os.path.join(analpath, "corecorrectedimg" + "." + "png")
imgpath= r"D:\OneDrive - UNSW\Image_Analysis\typesofParticles\NewAItraining\newmicroscope\dimer"
threshold=0.25
savepath=os.path.join(analpath, "clumpcorrectedimg" + "." + "png")
img_rgb=imbef[0]

images=load_images_from_folderhsv(imgpath)


mask = np.zeros(img_rgb.shape, dtype=np.uint8)
res=[[]];
template=images[2]
w, h = template.shape[0:2]
#big image converted to hsv format
img_hsv= cv.cvtColor(img_rgb,  cv.COLOR_RGB2HSV)
#the location minima identified with the template matching algorithm
 

#print(res)


mat_of_matching_results=cv.matchTemplate(img_hsv,template,cv.TM_SQDIFF_NORMED) 
#This is to get some details about the minimum but isn't actually used
(min_val, max_val, min_loc, max_loc) = cv.minMaxLoc(mat_of_matching_results)
print(min_val)
print(max_val)
print(min_loc)
#plt.imshow(img_rgb)
#plt.imshow(img_hsv)
locations_of_minimum = np.where( mat_of_matching_results <= ((max_val-min_val)/6)+min_val)
particle_count=[]
# a copy of the image is used so there aren't random boxes drawn over the image
scanimage=img_hsv.copy()
mask = np.zeros(scanimage.shape[:2], dtype=np.uint8)
#initialising the mask. The type must be the same
for pt in zip(*locations_of_minimum[::-1]):           
    try: 
        #if there is no predrawn box around the point sampled, add the point to a list of coordinates
        if mask[pt[1] + int(round(h/2)), pt[0] + int(round(w/2))] != 255:
            mask[pt[1]:pt[1]+h, pt[0]:pt[0]+w] = 255
            particle_count.append(pt)
            cv.rectangle(scanimage, pt, (pt[0] + w, pt[1] + h), (0,255,0), 1)
    except:
        z=1  

print(len(particle_count))
image_rgb=imbef[0].copy()
for pt in particle_count:
    image_rgb = cv.rectangle(image_rgb,pt, (pt[0] + w, pt[1] + h) , (255,255,255), -1)
plt.figure(figsize = (10,10))
plt.imshow(image_rgb)
plt.imshow(imbef[0], alpha=0.5)

In [ ]:
img_rgb=imbef[0]
imgpath=r"D:\OneDrive - UNSW\Image_Analysis\typesofParticles\NewAItraining\newmicroscope\clump"
images=load_images_from_folderhsv(imgpath)


mask = np.zeros(img_rgb.shape, dtype=np.uint8)
res=[[]];
template=images[2]
w, h = template.shape[0:2]
#big image converted to hsv format
img_hsv= cv.cvtColor(img_rgb,  cv.COLOR_RGB2HSV)
#the location minima identified with the template matching algorithm
 

#print(res)


mat_of_matching_results=cv.matchTemplate(img_hsv,template,cv.TM_SQDIFF_NORMED) 
#This is to get some details about the minimum but isn't actually used
(min_val, max_val, min_loc, max_loc) = cv.minMaxLoc(mat_of_matching_results)
print(min_val)
print(max_val)
print(min_loc)
#plt.imshow(img_rgb)
#plt.imshow(img_hsv)
locations_of_minimum = np.where( mat_of_matching_results <= ((max_val-min_val)/6)+min_val)
particle_count=checkfordoublecounting(img_hsv,locations_of_minimum)
print(particle_count)